In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

%matplotlib qt

# import module with layers
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model_module")

from adaptive_layer import *

In [2]:
tf.random.set_seed(5)
np.random.seed(5)

In [3]:
def illustrate_train(name, y_label, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    epochs = range(1, len(static_f)+1)
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    plt.title(name, fontsize="xx-large")

    ax.set_ylabel(y_label, fontsize="large")
    ax.set_xlabel('Epochs', fontsize="large")

    ax.plot(epochs, static_f, label='ReLU Function')
    ax.plot(epochs, adaptive_f, label='Swish Function')
    ax.plot(epochs, adaptive_nn, label='Adaptive NN')
    ax.plot(epochs, adaptive_conv_nn, label='Adaptive Conv NN')

    ax.legend()
    
    plt.grid()
    plt.show()

In [4]:
def illustrate_test(name, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    
    x = ["ReLU Function", "Swish Function", "Adaptive NN", "Adaptive Conv NN"]
    y = [static_f, adaptive_f, adaptive_nn, adaptive_conv_nn]
    
    plt.title(name, fontsize="xx-large")
    ax.set_ylabel("Accuracy", fontsize="large")
    
    ax.bar(x, y)
    
    plt.grid()
    plt.show()

In [5]:
workbook = xlsxwriter.Workbook('bc.xlsx')

In [6]:
def save_data(name, static_f_train, adaptive_f_train, adaptive_nn_train, adaptive_conv_nn_train, 
              static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test):
    
    epochs = len(static_f_train[0][:])
    worksheet = workbook.add_worksheet(name)
    
    worksheet.write(1, 1, name + " Training Loss")
    worksheet.write(1, 7, name + " Training Accuracy")
    worksheet.write(1, 13, name + " Test Accuracy")
    
    worksheet.write(2, 1, "Epochs")
    worksheet.write(2, 2, "ReLU Function")
    worksheet.write(2, 3, "Swish Function")
    worksheet.write(2, 4, "Adaptive NN")
    worksheet.write(2, 5, "Adaptive Conv NN")
    
    worksheet.write(2, 7, "Epochs")
    worksheet.write(2, 8, "ReLU Function")
    worksheet.write(2, 9, "Swish Function")
    worksheet.write(2, 10, "Adaptive NN")
    worksheet.write(2, 11, "Adaptive Conv NN")
    
    worksheet.write(2, 13, "ReLU Function")
    worksheet.write(2, 14, "Swish Function")
    worksheet.write(2, 15, "Adaptive NN")
    worksheet.write(2, 16, "Adaptive Conv NN")
    
    worksheet.write(3, 13, static_f_test)
    worksheet.write(3, 14, adaptive_f_test)
    worksheet.write(3, 15, adaptive_nn_test)
    worksheet.write(3, 16, adaptive_conv_nn_test)
    
    for i in range(0, epochs):
        worksheet.write(3+i, 1, i+1)
        worksheet.write(3+i, 2, static_f_train[0][i])
        worksheet.write(3+i, 3, adaptive_f_train[0][i])
        worksheet.write(3+i, 4, adaptive_nn_train[0][i])
        worksheet.write(3+i, 5, adaptive_conv_nn_train[0][i])
        
        worksheet.write(3+i, 7, i+1)
        worksheet.write(3+i, 8, static_f_train[1][i])
        worksheet.write(3+i, 9, adaptive_f_train[1][i])
        worksheet.write(3+i, 10, adaptive_nn_train[1][i])
        worksheet.write(3+i, 11, adaptive_conv_nn_train[1][i])
    
    workbook.close()

In [7]:
# dataset

data = pd.read_csv('../datasets/sonar_data.csv')
data = pd.DataFrame(data).to_numpy()
np.random.shuffle(data)

x = data[:, :-1]
y = data[:, -1]

x = np.array(x, dtype=np.float32)
y = np.array([out == 'M' for out in (y)], dtype=np.float32)

x_train, x_test = x[:160, :], x[160:, :]
y_train, y_test = y[:160], y[160:]

y_train = tf.constant(y_train, shape=(y_train.shape[0], 1))
y_test = tf.constant(y_test, shape=(y_test.shape[0], 1))

In [8]:
EPOCHS = 200
name = "Sonar Binary Classification"

# measurement repetitions
flag = measurements = 5

static_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_nn_hist = np.zeros(shape=[2, EPOCHS])
adaptive_conv_nn_hist = np.zeros(shape=[2, EPOCHS])

static_f_test = 0
adaptive_f_test = 0
adaptive_nn_test = 0
adaptive_conv_nn_test = 0

In [89]:
static_f_model = keras.Sequential([
    keras.layers.Dense(5, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [90]:
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(5, activation=tf.nn.swish),
    keras.layers.Dense(1, activation="sigmoid")
])

In [91]:
# Adaptive Layer
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(5, structure=[2, 1], inner_out_activation=tf.nn.sigmoid, skip_w=0.1),
    AdaptiveLayer(1, structure=[2, 1], inner_out_activation=tf.nn.sigmoid, skip_w=0.1)
])

In [92]:
# Adaptive Layer based on 1D Conv
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(5, structure=[2, 4], split=3, skip_w=0, noise=0, inner_out_activation=tf.nn.sigmoid),
    AdaptiveLayerConv(1, structure=[2, 4], skip_w=0, noise=0, inner_out_activation=tf.nn.sigmoid)
])

In [93]:
static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.003), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [94]:
adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.003), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [95]:
adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.003), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [96]:
adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.003), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [97]:
hist = static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6940 - accuracy: 0.5250
Epoch 2/200
3/3 [==============================] - 0s 849us/step - loss: 0.6863 - accuracy: 0.5625
Epoch 3/200
3/3 [==============================] - 0s 890us/step - loss: 0.6824 - accuracy: 0.5813
Epoch 4/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6795 - accuracy: 0.5813
Epoch 5/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6740 - accuracy: 0.5813
Epoch 6/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6688 - accuracy: 0.6125
Epoch 7/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6636 - accuracy: 0.6125
Epoch 8/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6587 - accuracy: 0.6438
Epoch 9/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6550 - accuracy: 0.6562
Epoch 10/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6502 - accuracy: 0.6562
Epoch 11/200
3/3 [=======

In [98]:
static_f_hist[0][:] += np.array(hist.history["loss"])/measurements
static_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [99]:
hist = adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
3/3 [==============================] - 0s 1ms/step - loss: 0.7104 - accuracy: 0.5125
Epoch 2/200
3/3 [==============================] - 0s 925us/step - loss: 0.6937 - accuracy: 0.5188
Epoch 3/200
3/3 [==============================] - 0s 908us/step - loss: 0.6834 - accuracy: 0.5250
Epoch 4/200
3/3 [==============================] - 0s 835us/step - loss: 0.6756 - accuracy: 0.5312
Epoch 5/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6707 - accuracy: 0.5750
Epoch 6/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6664 - accuracy: 0.5625
Epoch 7/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6618 - accuracy: 0.5875
Epoch 8/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6569 - accuracy: 0.6125
Epoch 9/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6511 - accuracy: 0.6375
Epoch 10/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6445 - accuracy: 0.6562
Epoch 11/200
3/3 [=====

In [100]:
adaptive_f_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [101]:
hist = adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6935 - accuracy: 0.4875
Epoch 2/200
3/3 [==============================] - 0s 964us/step - loss: 0.6931 - accuracy: 0.5063
Epoch 3/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6930 - accuracy: 0.5125
Epoch 4/200
3/3 [==============================] - 0s 3ms/step - loss: 0.6930 - accuracy: 0.5125
Epoch 5/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6927 - accuracy: 0.5125
Epoch 6/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6926 - accuracy: 0.5125
Epoch 7/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6924 - accuracy: 0.5125
Epoch 8/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6923 - accuracy: 0.5125
Epoch 9/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.5125
Epoch 10/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6917 - accuracy: 0.5125
Epoch 11/200
3/3 [=========

In [102]:
adaptive_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [103]:
hist = adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6965 - accuracy: 0.4875
Epoch 2/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6943 - accuracy: 0.4875
Epoch 3/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6934 - accuracy: 0.4875
Epoch 4/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6934 - accuracy: 0.5125
Epoch 5/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6929 - accuracy: 0.5125
Epoch 6/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6927 - accuracy: 0.5125
Epoch 7/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6927 - accuracy: 0.5125
Epoch 8/200
3/3 [==============================] - 0s 3ms/step - loss: 0.6926 - accuracy: 0.5125
Epoch 9/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6925 - accuracy: 0.5125
Epoch 10/200
3/3 [==============================] - 0s 3ms/step - loss: 0.6920 - accuracy: 0.5125
Epoch 11/200
3/3 [===========

In [104]:
adaptive_conv_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_conv_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [105]:
static_f_test += static_f_model.evaluate(x_test, y_test)[1]/measurements

2/2 [==============================] - 0s 2ms/step - loss: 0.4474 - accuracy: 0.7872


In [106]:
adaptive_f_test += adaptive_f_model.evaluate(x_test, y_test)[1]/measurements

2/2 [==============================] - 0s 2ms/step - loss: 0.5039 - accuracy: 0.7660


In [107]:
adaptive_nn_test += adaptive_nn_model.evaluate(x_test, y_test)[1]/measurements

2/2 [==============================] - 0s 2ms/step - loss: 0.3500 - accuracy: 0.8723


In [108]:
adaptive_conv_nn_test += adaptive_conv_nn_model.evaluate(x_test, y_test)[1]/measurements
flag -= 1
if(flag == 0):
    print("---------FINISH---------")

2/2 [==============================] - 0s 2ms/step - loss: 0.3696 - accuracy: 0.8936
---------FINISH---------


In [109]:
print("Static Function model\nTest accuracy : ", static_f_test)
print("\nAdaptive Function model\nTest accuracy : ", adaptive_f_test)
print("\nAdaptive NN model\nTest accuracy : ", adaptive_nn_test)
print("\nAdaptive Conv NN model\nTest accuracy : ", adaptive_conv_nn_test)

Static Function model
Test accuracy :  0.8085106611251831

Adaptive Function model
Test accuracy :  0.7957447052001954

Adaptive NN model
Test accuracy :  0.9021276712417603

Adaptive Conv NN model
Test accuracy :  0.8893617153167725


In [110]:
illustrate_train(name+" Training Loss", "Loss", static_f_hist[0][:], 
                 adaptive_f_hist[0][:], adaptive_nn_hist[0][:], adaptive_conv_nn_hist[0][:])

In [111]:
illustrate_train(name + " Training Accuracy", "Accuracy", static_f_hist[1][:], 
                 adaptive_f_hist[1][:], adaptive_nn_hist[1][:], adaptive_conv_nn_hist[1][:])

In [112]:
illustrate_test(name + " Test Accuracy", 
                static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)

In [113]:
save_data(name, static_f_hist, adaptive_f_hist, adaptive_nn_hist, 
          adaptive_conv_nn_hist, static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)